<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/03-Advanced_RedisSearch/03-Advanced_RedisSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RediSearch

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

This notebook is an adapted and simplified version of the RedisInsight QuickGuide "Working with Hashes".

For the full exterience we'd recommend installing Redis Insight and going through tutorial there.

https://redis.io/insight/

In [1]:
# Install the requirements
!pip install -q redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 6.2 MB/s eta 0:00:00


In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb focal main
Starting redis-stack-server, database path /var/lib/redis-stack


In [4]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

In [5]:
import redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
r.ping()

True

## Redis Hashes

Hash is a fundamental Redis datatype.


See full list of Redis HASH commands here: https://redis.io/commands/?group=hash




In [6]:
schools = [
    {"name":"Hall School",
     "description":"Spanning 10 states, this school award-winning curriculum includes a comprehensive reading system (from letter recognition and phonics to reading full-length books), as well as math, science, social studies, and even  philosophy.",
     "class":"independent",
     "type":"traditional",
     "address_city":"London",
     "address_street":"Manor Street",
     "students":342,
     "location":"51.445417, -0.258352"
     },
    {"name":"Garden School",
     "description":"Garden School is a new and innovative outdoor teaching and learning experience, offering rich and varied activities in a natural environment to children and families.",
     "class":"state","type":"forest; montessori",
     "address_city":"London",
     "address_street":"Gordon Street",
     "students":1452,
     "location":"51.402926, -0.321523",
     },

    {"name":"Gillford School",
     "description":"Gillford School is an inclusive learning centre welcoming people from all walks of life, here invited to step into their role as regenerative agents, creating new pathways into the future and inciting an international movement of cultural, land, and social transformation.",
     "class":"private",
     "type":"democratic; waldorf",
     "address_city":"Goudhurst",
     "address_street":"Goudhurst",
     "students":721,
     "location":"51.112685, 0.451076"
     },

     {
     "name":"Forest School",
     "description":"The philosophy behind Forest School is based upon the desire to provide young children with an education that encourages appreciation of the wide world in nature while achieving independence, confidence and high self-esteem. ",
     "class":"independent",
     "type":"forest; montessori; democratic",
     "address_city":"Oxford",
     "address_street":"Trident Street",
     "students":1200,
     "location":"51.781756, -1.123196"
     }
    ]
#load data in Redis as JSON
for id,school in enumerate(schools):
    #print(school)
    r.hset(f"school:{id}", mapping = school)

In [7]:
!redis-cli $REDIS_CONN keys 'school:*'
!echo
!redis-cli $REDIS_CONN HGETALL school:1



1) "school:2"
2) "school:3"
3) "school:1"
4) "school:0"

 1) "students"
 2) "1452"
 3) "address_city"
 4) "London"
 5) "address_street"
 6) "Gordon Street"
 7) "type"
 8) "forest; montessori"
 9) "name"
10) "Garden School"
11) "class"
12) "state"
13) "location"
14) "51.402926, -0.321523"
15) "description"
16) "Garden School is a new and innovative outdoor teaching and learning experience, offering rich and varied activities in a natural environment to children and families."


In [8]:
#!redis-cli $REDIS_CONN flushdb

## RediSearch

RediSearch adds the ability to query data in your HASH or JSON data structures, essentially turning Redis into the docuemnt database.

With RediSearch you declare indices once and then every database object matching the prefix, defined in the index would be automatically and in real time added to the index.

For the full list of RediSearch commands see: https://redis.io/commands/?group=search

Python documentation: https://redis-py.readthedocs.io/en/stable/redismodules.html#redisearch-commands

In [9]:
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    GeoField,
    VectorField
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search import reducers
schema = (
    TextField("name", as_name="name"),
    TextField("description", as_name="description"),
    TagField("address_city", as_name="city"),
    TagField("type", as_name="type", separator=";"),
    NumericField("students", as_name="students"),
    GeoField("location", as_name="location")
    )
r.ft("idx:schools").create_index(schema,
                    definition=IndexDefinition(prefix=["school:"],
                    index_type=IndexType.HASH)
                    )

b'OK'

In [10]:
#Retrieve index information
r.ft("idx:schools").info()


{'index_name': 'idx:schools',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'school:'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'name',
   b'attribute',
   b'name',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'description',
   b'attribute',
   b'description',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'address_city',
   b'attribute',
   b'city',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b','],
  [b'identifier',
   b'type',
   b'attribute',
   b'type',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b';'],
  [b'identifier', b'students', b'attribute', b'students', b'type', b'NUMERIC'],
  [b'identifier', b'location', b'attribute', b'location', b'type', b'GEO']],
 'num_docs': '4',
 'max_doc_id': '4',
 'num_terms': '130',
 'num_records': '158',
 'inverted_sz_mb': '0.00088214874267578125',
 'vector_index_sz_mb': '0',
 'total_inverted_index_blocks': '142',
 'offset_vectors_sz_mb'

In [11]:
import pandas as pd

#helper function to display results of redis.ft().search() as a dataframe
def display_ft(res):
  if res.total==0:
    print("No matches found")
  else:
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    display(res_df)

#helper function to translate aggregate result to dataframe and display it
#function is created with the help of ChatGPT: https://chat.openai.com/share/fc4e4ea5-d421-4aaf-a1b2-6fac02c96f20
def display_ft_agg(res):
  data = res.rows
  data = [[item.decode('utf-8') for item in sublist] for sublist in data]
  column_dict = {}
  for sublist in data:
      for i in range(0, len(sublist), 2):
          column_name = sublist[i]
          column_value = sublist[i + 1]
          column_dict.setdefault(column_name, []).append(column_value)
  df = pd.DataFrame(column_dict)
  display(df)

## Text search

You can run full text search queries on any field you marked to be indexed as `TEXT` or `TextField()` in Python.

To specify the specific field use `@field:value` syntax.

You can also do pattern matching, using `@field:val*`

In [12]:
#return the entire document
res=r.ft("idx:schools").search("nature")
display_ft(res)

,id,students,address_street,type,location,address_city,name,class,description
0,school:3,1200,Trident Street,forest; montessori; democratic,"51.781756, -1.123196",Oxford,Forest School,independent,The philosophy behind Forest School is based u...
1,school:1,1452,Gordon Street,forest; montessori,"51.402926, -0.321523",London,Garden School,state,Garden School is a new and innovative outdoor ...


In [13]:
#Full text search, return selected fields only
query=Query("nature") \
   .return_field("address_city", as_field="city") \
   .return_field("name", as_field="name")
res=r.ft("idx:schools").search(query)
display_ft(res)


,id,city,name
0,school:3,Oxford,Forest School
1,school:1,London,Garden School


## Search with multiple parameters
You can combine conditions on multiple fields using space as a logical AND or pipe `|` as logical OR.

In [14]:
# Perform a search for documents that have all of the tags (AND condition)
query=Query("@type:{forest} @type:{montessori}")
res=r.ft("idx:schools").search(query)
display_ft(res)


,id,students,address_street,type,location,address_city,name,class,description
0,school:1,1452,Gordon Street,forest; montessori,"51.402926, -0.321523",London,Garden School,state,Garden School is a new and innovative outdoor ...
1,school:3,1200,Trident Street,forest; montessori; democratic,"51.781756, -1.123196",Oxford,Forest School,independent,The philosophy behind Forest School is based u...


In [15]:
# Perform a search for documents that are either in Goudhurst or of type montessori (OR condition)
query=Query("(@city:{Goudhurst})|(@type:{montessori})")
res=r.ft("idx:schools").search(query)
display_ft(res)

,id,students,address_street,type,location,address_city,name,class,description
0,school:2,721,Goudhurst,democratic; waldorf,"51.112685, 0.451076",Goudhurst,Gillford School,private,Gillford School is an inclusive learning centr...
1,school:1,1452,Gordon Street,forest; montessori,"51.402926, -0.321523",London,Garden School,state,Garden School is a new and innovative outdoor ...
2,school:3,1200,Trident Street,forest; montessori; democratic,"51.781756, -1.123196",Oxford,Forest School,independent,The philosophy behind Forest School is based u...


## TAG, Numeric and Geo search

For TAG fields - use `@field:{value}` syntax.

Geo radius matches on geo fields with the syntax `@field:[lon lat radius {m|km|mi|ft}]`

Numeric ranges look like `@students:[0, 10000]` with square brackets used as inclusive (less or equal, greater or equal) and regular braces for exclusive (less or greater).

In [16]:
#Geolocation search
query = Query('@location:[51.3 0.32 30 km]')
res = r.ft('idx:schools').search(query)
display_ft(res)

,id,students,address_street,type,location,address_city,name,class,description
0,school:2,721,Goudhurst,democratic; waldorf,"51.112685, 0.451076",Goudhurst,Gillford School,private,Gillford School is an inclusive learning centr...


In [17]:
#Combining Tag and Geo conditions
query=Query('@city:{London} @students:[0, 10000]') \
   .return_field("address_city", as_field="city") \
   .return_field("name", as_field="name") \
   .return_field("students", as_field="students")
res=r.ft("idx:schools").search(query)
display_ft(res)



,id,city,name,students
0,school:1,London,Garden School,1452
1,school:0,London,Hall School,342


## Aggregations
Aggregations are a way to process the results of a search query, group, sort and transform them - and extract analytic insights from them. Much like aggregation queries in other databases and search engines, they can be used to create analytics reports, or perform Faceted Search style queries.

For example, we can group schools by city and count schools per group, giving us the number of schools per city. Or we could group by school class (independent/state) and see the average number of students per group.

In [18]:
#Perform aggregation by city and count number of records
request = AggregateRequest(f'*').group_by('@city', reducers.count().alias('count'))
res = r.ft("idx:schools").aggregate(request)
display_ft_agg(res)



,city,count
0,Goudhurst,1
1,London,2
2,Oxford,1


In [19]:
#Perform aggregation by city and count number of students
request = AggregateRequest(f'*').group_by('@city', reducers.sum('@students').alias('students_count'))
res = r.ft("idx:schools").aggregate(request)
display_ft_agg(res)

,city,students_count
0,Goudhurst,721
1,London,1794
2,Oxford,1200
